In [1]:
# # --- INSTALL DEPENDENCIES ---
# # This checks if libraries are installed and installs them if missing.
# # The '-q' flag keeps the output clean (quiet mode).

# import sys

# # Standard libraries for Deep Learning & Plotting
# !{sys.executable} -m pip install -q torch matplotlib graphviz

# print("Libraries installed and ready!")
# https://tinyurl.com/ds2026nn

In [2]:
import math


class Value:
    def __init__(self, data, _children=(), _op="", label=""):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data:.4f}, grad={self.grad:.4f})"

    # --- BASIC ARITHMETIC ---
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), "+")

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), "*")

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def __pow__(self, n):  # Support for x**2, etc.
        assert isinstance(n, (int, float))
        out = Value(self.data**n, (self,), f"**{n}")

        def _backward():
            self.grad += (n * self.data ** (n - 1)) * out.grad

        out._backward = _backward
        return out

    # --- ACTIVATIONS (Tweak these if asked for ReLU/Sigmoid) ---
    def tanh(self):
        x = self.data
        t = (math.exp(2 * x) - 1) / (math.exp(2 * x) + 1)
        out = Value(t, (self,), "tanh")

        def _backward():
            self.grad += (1 - t**2) * out.grad

        out._backward = _backward
        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), "ReLU")

        def _backward():
            self.grad += (1.0 if out.data > 0 else 0) * out.grad

        out._backward = _backward
        return out

    def sigmoid(self):
        x = self.data
        s = 1 / (1 + math.exp(-x))
        out = Value(s, (self,), "sigmoid")

        def _backward():
            # Sigmoid derivative: s * (1 - s)
            self.grad += (s * (1 - s)) * out.grad

        out._backward = _backward
        return out

    # --- BOILERPLATE FOR PYTHON OPS ---
    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __radd__(self, other):
        return self + other

    def __rmul__(self, other):
        return self * other

    def __truediv__(self, other):
        return self * (other**-1)

    # --- THE BACKPROP TRIGGER ---
    def backward(self):
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [3]:
# 1. DEFINE THE INPUT VALUES
a = Value(2.0, label="a")
b = Value(-3.0, label="b")
c = Value(10.0, label="c")

# 2. DEFINE THE EQUATION (The Forward Pass)
# If the equation changes to L = (a * b) + c, write it exactly like this:
e = a * b
e.label = "e"
L = e + c
L.label = "L"

# 3. TRIGGER BACKPROP
L.backward()

# 4. VIEW RESULTS
print(f"L data: {L.data}")  # The output value
print(f"a grad: {a.grad}")  # dL/da
print(f"b grad: {b.grad}")  # dL/db

L data: 4.0
a grad: -3.0
b grad: 2.0


In [4]:
import random


class Module:
    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0.0


class Neuron(Module):
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        # Change .tanh() to .relu() here if requested
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        return act.tanh()

    def parameters(self):
        return self.w + [self.b]


class Layer(Module):
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]


class MLP(Module):
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i + 1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [5]:
# 1. SETUP DATA
xs = [[2.0, 3.0, -1.0], [3.0, -1.0, 0.5], [0.5, 1.0, 1.0], [1.0, 1.0, -1.0]]
ys = [1.0, -1.0, -1.0, 1.0]

# 2. DEFINE MODEL (Inputs, [Hidden Layers], Outputs)
model = MLP(3, [4, 4, 1])

# 3. TRAINING LOOP
for k in range(50):
    # Forward pass
    ypred = [model(x) for x in xs]

    # LOSS: Change to (yout - ygt)**2 for MSE         abs(yout.data - ygt) for MAE
    loss = sum((yout - ygt) ** 2 for ygt, yout in zip(ys, ypred))
    # loss = sum((yout - ygt) if yout.data > ygt else (ygt - yout) for ygt, yout in zip(ys, ypred))
    # Backward pass
    model.zero_grad()
    loss.backward()

    # Update (Gradient Descent)
    lr = 0.1  # Change learning rate here
    for p in model.parameters():
        p.data += -lr * p.grad

    if k % 10 == 0:
        print(f"Step {k} | Loss: {loss.data:.4f}")

Step 0 | Loss: 6.0970
Step 10 | Loss: 0.0284
Step 20 | Loss: 0.0144
Step 30 | Loss: 0.0095
Step 40 | Loss: 0.0070


In [ ]:
# import torch

# # 1. SETUP DATA
# # requires_grad=False for data (we don't train input data)
# xs = torch.tensor([[2.0, 3.0, -1.0], [3.0, -1.0, 0.5], [0.5, 1.0, 1.0], [1.0, 1.0, -1.0]])
# ys = torch.tensor([1.0, -1.0, -1.0, 1.0])

# # 2. INITIALIZE WEIGHTS
# # generator=g ensures randomness is reproducible
# g = torch.Generator().manual_seed(2147483647)

# # We create tensors for weights. requires_grad=True is CRITICAL.
# # This replaces your 'Value' class logic.
# W1 = torch.randn((3, 4), generator=g, requires_grad=True)
# b1 = torch.randn(4, generator=g, requires_grad=True)
# W2 = torch.randn((4, 4), generator=g, requires_grad=True)
# b2 = torch.randn(4, generator=g, requires_grad=True)
# W3 = torch.randn((4, 1), generator=g, requires_grad=True)
# b3 = torch.randn(1, generator=g, requires_grad=True)

# parameters = [W1, b1, W2, b2, W3, b3]

# # 3. TRAINING LOOP
# for k in range(50):
    
#     # Forward Pass (The Math)
#     # .tanh() is built into PyTorch tensors
#     l1 = torch.tanh(xs @ W1 + b1) 
#     l2 = torch.tanh(l1 @ W2 + b2)
#     ypred = torch.tanh(l2 @ W3 + b3)
    
#     # Loss (Mean Squared Error)
#     loss = ((ypred.squeeze() - ys)**2).sum()
    
#     # Backward Pass (The "One Function")
#     # This does EVERYTHING your topological sort and _backward did.
#     for p in parameters:
#         p.grad = None # Reset gradients to 0
#     loss.backward()
    
#     # Update (Gradient Descent)
#     lr = 0.1
#     # We must wrap updates in 'no_grad()' so PyTorch doesn't track this math
#     with torch.no_grad():
#         for p in parameters:
#             p += -lr * p.grad

#     if k % 10 == 0:
#         print(f"Step {k} | Loss: {loss.item():.4f}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. DATA
xs = torch.tensor([[2.0, 3.0, -1.0], [3.0, -1.0, 0.5], [0.5, 1.0, 1.0], [1.0, 1.0, -1.0]])
ys = torch.tensor([[1.0], [-1.0], [-1.0], [1.0]])

# 2. MODEL
# nn.Sequential stacks layers for you
model = nn.Sequential(
    nn.Linear(3, 4), # Matrix multiplication + bias automatically included
    nn.Tanh(),
    nn.Linear(4, 4),
    nn.Tanh(),
    nn.Linear(4, 1),
    nn.Tanh()
)

# 3. OPTIMIZER
# SGD handles the "p.data += -lr * p.grad" part for you
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 4. LOOP
for k in range(50):
    # Forward
    ypred = model(xs)
    loss = ((ypred - ys)**2).sum()
    
    # Backward
    optimizer.zero_grad() # Replaces p.grad = None
    loss.backward()
    optimizer.step()      # Updates weights
    
    if k % 10 == 0:
        print(f"Step {k} | Loss: {loss.item():.4f}")